# parte 1
pegar os links de cada um dos conjuntos

In [1]:
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
driver = webdriver.Chrome(options=chrome_options)
driver.set_page_load_timeout(60)
url = 'https://www.quintoandar.com.br/alugar/imovel/sao-paulo-sp-brasil'

In [2]:
driver.get(url)
driver.implicitly_wait(10)

wait = WebDriverWait(driver, timeout=5)

In [3]:
horas = []

for i in range(2):
    try:
        horas = driver.find_elements(By.XPATH, '//main/section[2]/div/div')
        wait.until(EC.element_to_be_clickable((By.XPATH, '//main/section[2]/div/div[last()-2]/button')))
        horas[-3].click()
        wait.until(EC.staleness_of(horas[-3]))

    except Exception as err:
        print(f'\nse fudeu em {i}: {err = }\n')

        if isinstance(err, StaleElementReferenceException):
            print("Attempting to recover from StaleElementReferenceException")
            wait.until(EC.element_to_be_clickable((By.XPATH, '//main/section[2]/div/div[last()-2]/button')))
        else:
            raise err

    print(f'list size on iteration {i}: {len(horas)}')

list size on iteration 0: 16
list size on iteration 1: 28


In [4]:
links = []
# os divs de interesse vao desde o segundo ate o anteantepenultimo
for i, div in enumerate(horas[1:-4]):
    current_link = div.find_element(by=By.TAG_NAME, value='a').get_property('href')
    print(f'link at iteration {i} {current_link}')
    links.append(current_link)

link at iteration 0 https://www.quintoandar.com.br/imovel/893693970/alugar/kitnet-1-quarto-indianopolis-sao-paulo
link at iteration 1 https://www.quintoandar.com.br/imovel/894482097/alugar/apartamento-2-quartos-vila-santa-catarina-sao-paulo
link at iteration 2 https://www.quintoandar.com.br/imovel/894482910/alugar/kitnet-1-quarto-centro-sao-paulo
link at iteration 3 https://www.quintoandar.com.br/imovel/892950505/alugar/casa-2-quartos-sapopemba-sao-paulo
link at iteration 4 https://www.quintoandar.com.br/imovel/894483697/alugar/apartamento-2-quartos-vila-penteado-sao-paulo
link at iteration 5 https://www.quintoandar.com.br/imovel/892920653/alugar/casa-1-quarto-vila-parque-jabaquara-sao-paulo
link at iteration 6 https://www.quintoandar.com.br/imovel/894410806/alugar/casa-2-quartos-jardim-jau-zona-leste-sao-paulo
link at iteration 7 https://www.quintoandar.com.br/imovel/893352003/alugar/apartamento-2-quartos-jurubatuba-sao-paulo
link at iteration 8 https://www.quintoandar.com.br/imovel/8

# parte 2
pra cada conjunto, obter informaçoes de interess
e

In [11]:
xpaths = {

    'suite_area'    : '//main/section/div/div[1]/div/div[3]/section/div/div[3]/div/div/div[1]/div/div/p',
    'street'        : '//main/section/div/div[1]/div/div[2]/div/div/div/div[1]/div/h4',
    'neighborhood'  : '//main/section/div/div[1]/div/div[2]/div/div/div/div[1]/small',
    'condominium'   : '//main/section/div/div[1]/div/div[3]/section/div/div[2]/div/ul/li[2]/div/div/p',
    'tax'           : '//main/section/div/div[1]/div/div[3]/section/div/div[2]/div/ul/li[3]/div/div/p',
    'asking_price'  : '//main/section/div/div[1]/div/div[3]/section/div/div[2]/div/ul/li[1]/div/div/p'
}

In [16]:
import pandas as pd


delta_y = 800
stock = []
for link in links[0:2]:
    driver.get(link)
    driver.implicitly_wait(10)
    ActionChains(driver) \
        .scroll_by_amount(0, delta_y) \
        .perform()
    
    stock.append(
        {
            key: driver.find_element(by=By.XPATH, value=xpath).text for key, xpath in xpaths.items()
        } 
    )

In [18]:
pd.DataFrame(stock)

,suite_area,street,neighborhood,condominium,tax,asking_price
0,25 m²,Alameda Iraé,"Indianópolis, São Paulo",R$ 483,R$ 0,R$ 3.100
1,49 m²,Rua Doutor Djalma Pinheiro Franco,"Vila Santa Catarina, São Paulo",R$ 532,R$ 107,R$ 3.200
